# Deployment

Various deployment flows using tools

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 26/01/2026   | Martin | Created   | Deployment notebook created | 
| 30/01/2026   | Martin | Update | Completed deployment flow | 

# Content

* [Mlflow w/ Docker](#mlflow-w-docker)
* [Batch Deployment](#batch-deployment)

# Mlflow w/ Docker

Using Mlflow registry with Docker containers for serving

In [45]:
import os
import mlflow
from dotenv import dotenv_values
config = dotenv_values("../.env")

os.environ["AWS_ACCESS_KEY_ID"] = config["MLFLOW_USER"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["MLFLOW_PASSWORD"]
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://127.0.0.1:9000"
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [46]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

from mlflow.models import infer_signature

In [47]:
mlflow.set_experiment("taxi-prediction")

2026/01/26 15:15:31 INFO mlflow.tracking.fluent: Experiment with name 'taxi-prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/1', creation_time=1769411731192, experiment_id='1', last_update_time=1769411731192, lifecycle_stage='active', name='taxi-prediction', tags={}>

In [48]:
def read_dataframe(filename: str):
  df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.dt.total_seconds() / 60
  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
  return df


def prepare_dictionaries(df: pd.DataFrame):
  df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
  categorical = ['PU_DO']
  numerical = ['trip_distance']
  dicts = df[categorical + numerical].to_dict(orient='records')
  return dicts

In [54]:
df_train = read_dataframe('../data/raw/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/raw/green_tripdata_2025-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [56]:
with mlflow.start_run():
  params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
  cols = ['PU_DO', 'trip_distance']
  mlflow.log_params(params)

  model = RandomForestRegressor(**params, n_jobs=-1)

  model.fit(df_train[cols], y_train)
  y_pred = model.predict(df_val[cols])

  rmse = mean_squared_error(y_pred, y_val)
  print(params, rmse)
  mlflow.log_metric('rmse', rmse)

  signature = infer_signature(df_train[cols].iloc[[0]], y_train[0])

  mlflow.sklearn.log_model(model, name="duration-pred", input_example=df_train[cols].iloc[[0]], signature=signature)

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 29.065911785392863


🏃 View run chill-koi-886 at: http://127.0.0.1:5000/#/experiments/1/runs/77a718179132447e82f6ea35184e4862
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [58]:
model_load = mlflow.sklearn.load_model("runs:/77a718179132447e82f6ea35184e4862/duration-pred")

In [59]:
df_train.iloc[0]

VendorID                                   2
lpep_pickup_datetime     2025-01-01 00:03:01
lpep_dropoff_datetime    2025-01-01 00:17:12
store_and_fwd_flag                         N
RatecodeID                               1.0
PULocationID                              75
DOLocationID                             235
passenger_count                          1.0
trip_distance                           5.93
fare_amount                             24.7
extra                                    1.0
mta_tax                                  0.5
tip_amount                               6.8
tolls_amount                             0.0
ehail_fee                                NaN
improvement_surcharge                    1.0
total_amount                            34.0
payment_type                             1.0
trip_type                                1.0
congestion_surcharge                     0.0
cbd_congestion_fee                       0.0
duration                           14.183333
PU_DO     

In [60]:
model.predict(df_val[cols].iloc[[0]])

array([5.17751339])

In [44]:
df_val[cols].iloc[[0]]

,PU_DO,trip_distance
0,166_41,0.65


---

# Batch Deployment

In [ ]:
import pickle
import pandas as pd
import uuid

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
import os
import mlflow
from dotenv import dotenv_values
config = dotenv_values("../.env")

os.environ["AWS_ACCESS_KEY_ID"] = config["MLFLOW_USER"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["MLFLOW_PASSWORD"]
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://127.0.0.1:9000"
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment('taxi-prediction')

<Experiment: artifact_location='s3://mlflow/1', creation_time=1769411731192, experiment_id='1', last_update_time=1769411731192, lifecycle_stage='active', name='taxi-prediction', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [4]:
year = 2021
month = 2
taxi_type = 'green'

input_file = f"../data/raw/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

RUN_ID = os.getenv("RUN_ID", "692e91a6377b47bbb25c729a7bc74a8c")
run_uri = f"runs:/{RUN_ID}/dur_pred"
model = mlflow.pyfunc.load_model(run_uri)

In [ ]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: s3://mlflow/1/models/m-53c610106db2462fbfdb31599c6e3ee2/artifacts
  flavor: mlflow.sklearn
  run_id: 692e91a6377b47bbb25c729a7bc74a8c

In [ ]:
def load_model(run_id):
  run_uri = f"runs:/{run_id}/dur_pred"
  model = mlflow.pyfunc.load_model(run_uri)
  return model

def generate_uuids(n):
  ride_ids = []
  for i in range(n):
    ride_ids.append(str(uuid.uuid4()))
  return ride_ids

def read_dataframe(filename: str):
  df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.dt.total_seconds() / 60
  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
  return df


def prepare_dictionaries(df: pd.DataFrame):
  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)

  df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

  categorical = ['PU_DO']
  numerical = ['trip_distance']
  dicts = df[categorical + numerical].to_dict(orient='records')
  return dicts

In [14]:
# Create the results dataframe
def apply_model(input_file, run_id, output_file):
  df = read_dataframe(input_file)
  dicts = prepare_dictionaries(df)

  
  model = load_model(run_id)
  y_pred = model.predict(dicts)

  df_result = pd.DataFrame()
  df_result['ride_id'] = df['ride_id']
  df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
  df_result['PULocationID'] = df['PULocationID']
  df_result['DOLocationID'] = df['DOLocationID']
  df_result['actual_duration'] = df['duration']
  df_result['predicted_duration'] = y_pred
  df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
  df_result['model_version'] = run_id
  
  df_result.to_parquet(output_file, index=False)

In [ ]:
df_result

,ride_id,lpep_pickup_datetime,PULocationID,DOLocationID,actual_duration,predicted_duration,diff
0,b5bb0005-80e7-4a7b-9b23-6fb70d717028,2025-02-01 00:12:15,166,41,3.550000,5.178162,-1.628162
1,55620098-d49f-42db-bacb-2aa444255a49,2025-01-31 23:57:05,255,161,27.316667,28.111620,-0.794953
2,c05dd638-5d93-45af-b067-c491f91d4e28,2025-02-01 00:24:26,75,182,25.466667,24.404696,1.061971
3,5a003220-5e17-45cb-a716-108f2251439a,2025-02-01 00:17:15,97,209,8.683333,13.930882,-5.247549
4,f3874bf1-f144-487d-9ca3-e60e10c047b2,2025-02-01 00:17:36,7,223,9.000000,9.270449,-0.270449
...,...,...,...,...,...,...,...
46616,f77df1a8-23fb-4502-a424-125031a4bd5a,2025-02-28 22:35:00,181,161,35.000000,30.634552,4.365448
46617,b24bfc43-ef40-4f1c-89a7-e0d5b89fc453,2025-02-28 23:40:00,166,141,15.000000,20.178517,-5.178517
46618,775d34be-c6f7-4ca6-8a0a-9dddd49ed89d,2025-02-28 23:34:00,41,48,14.000000,19.927118,-5.927118
46619,3bba8aaa-1658-4807-bd67-0eeaddddc32b,2025-02-28 23:52:00,75,140,13.000000,15.699101,-2.699101


In [ ]:
%load_ext watermark
%watermark